In [7]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
from torch import nn
from torch.autograd import Variable

In [9]:
Xs = pd.read_csv('../data/sets/test-all-feat-from-kernel-repro.csv', nrows=1000000).drop('object_id', axis=1)
Xs = np.nan_to_num(Xs.values)
Xs = MinMaxScaler().fit_transform(Xs)
print(Xs.shape)
Xs.head()

(1000000, 68)


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [10]:
learning_rate = 0.05
num_epochs = 500

In [11]:
class autoencoder(nn.Module):
    def __init__(self, input_size, latent_size):
        super(autoencoder, self).__init__()

        assert input_size > latent_size + 2
        
        self.input_size = input_size
        self.latent_size = latent_size
        self.mid_size = (input_size + latent_size) // 2 

        self.encoder = nn.Sequential(
            nn.Linear(input_size, self.mid_size),
            nn.ReLU(True),
            nn.Linear(self.mid_size, latent_size)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_size, self.mid_size),
            nn.ReLU(True),
            nn.Linear(self.mid_size, input_size),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [12]:
model = autoencoder(input_size=Xs.shape[1],
                    latent_size=20
                   ).cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate,
                             weight_decay=1e-5)

RuntimeError: CUDA error: unspecified launch failure

In [6]:

for epoch in range(num_epochs):
    for X in [Xs]:
        X = Variable(torch.tensor(X.astype(np.float32))).cuda()
        # ===================forward=====================
        output = model(X)
        loss = criterion(output, X)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    if epoch % 10 == 0:
        print('epoch [{}/{}], loss:{:.4f}'
              .format(epoch+1, num_epochs, loss.data.item()))

epoch [1/500], loss:7226637347192832.0000
epoch [11/500], loss:1224039506903040.0000
epoch [21/500], loss:270357923627008.0000
epoch [31/500], loss:106407756038144.0000
epoch [41/500], loss:36106485628928.0000
epoch [51/500], loss:11862209986560.0000
epoch [61/500], loss:3911907803136.0000
epoch [71/500], loss:1056141148160.0000
epoch [81/500], loss:765299589120.0000
epoch [91/500], loss:201385590784.0000
epoch [101/500], loss:49225560064.0000
epoch [111/500], loss:18047805440.0000
epoch [121/500], loss:12467585024.0000
epoch [131/500], loss:8669190144.0000
epoch [141/500], loss:441104728064.0000
epoch [151/500], loss:178327863296.0000
epoch [161/500], loss:11322895892480.0000
epoch [171/500], loss:2760092155904.0000
epoch [181/500], loss:741086330880.0000
epoch [191/500], loss:781482131456.0000
epoch [201/500], loss:4145410304.0000
epoch [211/500], loss:99462537216.0000
epoch [221/500], loss:15280147456.0000
epoch [231/500], loss:1550161024.0000
epoch [241/500], loss:3191077376.0000
e

KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), './conv_autoencoder.pth')